Cluster for Papermill

In [1]:
import os

# api_key = ""
# project = ""
# region = ""
# authorizationToken = f"{os.environ['TEST_ACTIVATION_TOKEN']}" if os.getenv('TEST_ACTIVATION_TOKEN') else ""

# dataset_id = "basic_subclustering"
# clusteringType = "community-detection"
# cutoff = 0.75
# encode_type = "text"
# vector_fields = ["product_title_clip_vector_"]
# n_clusters = 10


token = ""

In [3]:
# because we don't know how to set env variables
try:
    import os
    os.environ["WORKFLOW_ID"] = job_id
except Exception as e:
    print(e)
    pass

name 'job_id' is not defined


In [5]:

## Instantiate client ###
from relevanceai import Client

In [6]:
from relevanceai.utils import decode_workflow_token 

config = decode_workflow_token(token)
authorizationToken = config['authorizationToken']
client = Client(token=config['authorizationToken'])

Connecting to ap-southeast-2...
Welcome to RelevanceAI. Logged in as 6e8374136fcf2cafbbd9.


In [9]:
dataset_id: str = config['dataset_id']
n_clusters: int = int(config.get('n_clusters', 100))
clusteringType = config['clusteringType']
vector_fields: list = config['vector_fields']
if isinstance(vector_fields, str):
    vector_fields = [vector_fields]

In [ ]:
## Checking valid vector field ###
for v in vector_fields:
  if not '_vector_'in v:
    raise ValueError(f"'{v}' is not a valid vector field")

ds = client.Dataset(dataset_id)
filters = config.get('filters', None)
try:
  if clusteringType == 'community-detection':
    cluster_method  = "community_detection"
    ds.cluster(
        model=cluster_method,
        cluster_config={"threshold": cutoff},
        vector_fields=vector_fields,
        filters=filters,
    )
  elif clusteringType == 'kmeans':
    if ds.shape[0] < 10000:
      from sklearn.cluster import KMeans
      cluster_method = 'kmeans'
      model = KMeans(n_clusters=n_clusters, random_state=42)
      alias = f"{cluster_method}-{n_clusters}"
      ds.cluster(
          model=model,
          cluster_config={"n_clusters": n_clusters, "random_state": 42},
          vector_fields=vector_fields,
          alias=alias,
          filters=filters,
      )
    else:
      cluster_method = 'minibatchkmeans'
      from sklearn.cluster import MiniBatchKMeans
      cluster_method = 'kmeans'
      model = MiniBatchKMeans(n_clusters=n_clusters, random_state=42)
      alias = f"{cluster_method}-{n_clusters}"
      n_clusters = int(n_clusters)
      chunksize = n_clusters + 100
      ds.batch_cluster(
          model=model,
          cluster_config={"n_clusters": n_clusters, "random_state": 42},
          vector_fields=vector_fields,
          alias=alias,
          filters=filters,
          transform_chunksize=chunksize
      )
  elif clusteringType == "kmedoids":
    from sklearn_extra.cluster import KMedoids
    cluster_method = "kmedoids"
    model = KMedoids(n_clusters=n_clusters, random_state=42, init="k-medoids++")
    alias = f"{cluster_method}-{n_clusters}"
    ds.cluster(
        model=model,
        vector_fields=vector_fields,
        alias=alias,
        filters=filters,
    )
except Exception as e:
  raise ValueError(f'{e}')

print("Finished clustering your data, you may close this window.")

ds.update_field_children(
    field=vector_fields[0],
    field_children=[".".join(["_cluster_", vector_fields[0], alias])],
    category="cluster"
)

client.workflows.status(
  workflow_id=job_id,
  status="complete",
  workflow_name="Clustering",
  metadata={},
  additional_information="",
)